In [63]:
# takes two list of urls as input, applies models and generates score of being fraud and clone,
# returns two lists of dictionaries

from ipynb.fs.full.Clone_Logo import clone_logo
from ipynb.fs.full.Clone_Desc import clone_desc
from ipynb.fs.full.Fraud_Desc import fraud_desc
from ipynb.fs.full.Fraud_Vers import fraud_vers
from ipynb.fs.full.Fraud_Reply import fraud_reply
from ipynb.fs.full.Fraud_Dev import fraud_dev
from ipynb.fs.full.Fraud_Metrics import fraud_metrics
from ipynb.fs.full.Fraud_Review import fraud_review
from urllib.error import HTTPError
from google_play_scraper import exceptions

def cal(rs,met,ver,des,dev,rep):
    dict = {'Review_Sentiment':0.25*rs,'Metrics':0.20*met,'Version':0.15*ver,'Description':0.20*des,'Developer':0.10*dev, 'Reply_Content':0.05*rep}
    x = dict.values()
    fval = sum(x)
    max_ele = max(x)
    for key,value in dict.items():
        if(value == max_ele):
            return key, fval
    return 0,0


def driver_2(id_inputs, id_database):

    flist = []
    clist = []

    fraud_version_list = fraud_vers(id_inputs)
    fraud_reply_list = fraud_reply(id_inputs)
    i = 0

    for app_id in id_inputs:

        if(app_id in id_database):
            return 0,0

        ifdict = dict()
        icdict = dict()   

        dpath1 = 'buffers/logo1.png'
        dpath2 = 'buffers/logo2.png'

        
        try:
            cval_logo, cval_logo_id = clone_logo(app_id,id_database,dpath1,dpath2)
            cval_desc, cval_desc_id = clone_desc(app_id,id_database)

            fval_review = fraud_review(app_id)
            fval_metrics = fraud_metrics(app_id)
            fval_verion = fraud_version_list[i]
            fval_reply = fraud_reply_list[i]
            fval_description = fraud_desc(app_id)
            fval_developer = fraud_dev(app_id,id_database)
        

            # clone:

            cflag = 1
            if cval_logo == 0 and cval_desc != 0 :
                cval_total = cval_desc
                cval_max = 'Matching Description'
                cval_id = cval_desc_id
            elif cval_desc == 0 and cval_logo != 0:
                cval_total = cval_logo
                cval_max = 'Matching Logo'
                cval_id = cval_logo_id
            elif cval_desc != 0 and cval_logo != 0:
                if cval_logo > cval_desc:
                    cval_total = cval_logo
                    cval_max = 'Matching Logo'
                    cval_id = cval_logo_id
                else:
                    cval_total = cval_desc
                    cval_max = 'Matching Description'
                    cval_id = cval_desc_id
            else:
                cval_id = 0
                cval_max = 0
                cval_total = 0
                cflag = 0

            icdict['App_Id_Clone'] = app_id
            icdict['App_Id_Real'] = cval_id
            icdict['Reason'] = cval_max
            icdict['Attribute'] = cval_total

            # fraud:        

            fval_max, fval_total = cal(fval_review,fval_metrics,fval_verion,fval_description,fval_developer,fval_reply)

            fflag = 1

            ifdict['App_Id'] = app_id
            ifdict['Reason'] = fval_max 
            ifdict['Probability'] = fval_total

            if fval_total < 0.6:
                fflag = 0

            if fflag:
                flist.append(ifdict)
                
            if cflag:
                clist.append(icdict)

        except exceptions.NotFoundError as e:
            print('Some Apps not found, please check database and input : NotFoundError')
            i += 1
            continue
        except KeyError as e:
            print('Something went wrong with Keys : KeyError')
            i += 1
            continue

        i += 1

    return flist, clist

In [64]:
if __name__ == "__main__":
    id_database = ['com.nextbillion.groww','com.tilicho.lendpal','com.branch_international.branch.branch_demo_android','com.finmapp.fininfinity','com.mymoneymantra.customer.app','io.attabot.app.paymeindia','com.games24x7.rummycircle.rummy','com.worldlineindia.ubi.merchantbqr','com.infrasoft.uboi','com.canarabank.mobility','com.dcb','com.revolut.junior','com.financialexpress.android','com.indianexpress.android','com.digilocker.android','com.mpokket.app','com.google.android.apps.tachyon','com.voldspeed.fast','com.snapchat.android','com.fengshenidle.fangzhi','com.leyoule.syhj','com.goodfun.mxy','com.rainbowgames.solitaire.google','com.anfn.jqioe.teenpatti','com.online.real.indian.cards.palacerummy','com.rotoql.betql','com.paytmmall','rummy.fantasy.firstgames','com.paytmmoney','com.bookmark.money','com.trueadviserhe.personalloan','com.tilicho.lendpal','com.iiflfinance.mymoney','tech.fplabs.score','com.epifi.paisa','com.nu.production','com.iexceed.appzillon.ippbMB','com.mnc.mbanking','com.bankofbaroda.mconnect','com.teenpatti.hd.gold.rummy','com.teenpatti.hd.gold','com.games24x7.ultimaterummy.playstore','com.tradewill.online','com.coindcx.btc','com.fusionmedia.investing','com.sbi.mf','com.forex.markets.app.android','com.kreditbee.android', 'com.citrus.citruspay', 'org.altruist.BajajExperia', 'com.gopaysense.android.boost', 'com.naviapp', 'com.bharatpe.app', 'com.kreditzy.android', 'com.moneytap.bnpl.app', 'com.stashfin.android', 'com.avail.easyloans.android', 'in.rebase.app', 'com.rupeek.customer', 'com.rupeeredee.app', 'in.zestmoney.android.zestlife', 'in.amazon.mShop.android.shopping', 'com.freecharge.android', 'com.axis.mobile', 'com.msf.kbank.mobile', 'com.upi.axispay', 'com.sbi.SBIFreedomPlus', 'com.sbi.lotusintouch', 'com.csam.icici.bank.imobile', 'com.dreamplug.androidapp', 'indwin.c3.shareapp', 'com.vaibhavkalpe.android.khatabook', 'money.jupiter', 'in.epaylater.android.retailer.generic', 'co.tslc.cashe.android', 'com.habile.cloudbankin.digimoney', 'com.moneyloji.loanservice', 'com.app.buenofinance', 'com.loaney.app', 'ng.com.fairmoney.fairmoney', 'com.flipkart.shopsy', 'com.flipkart.android', 'com.myntra.android', 'com.ril.ajio', 'com.meesho.supply', 'com.fsn.nds', 'com.snapdeal.main', 'com.tul.tatacliq', 'com.shopclues', 'live.video.shopping.simsim', 'mobile.qoo10.qstlin', 'booksfortune.bookchor', 'com.tatadigital.tcp', 'com.olx.southasia', 'com.jpl.jiomart', 'in.upstox.app', 'com.kotak.kst', 'com.dhan.live', 'com.zerodha.kite3', 'com.dream11.fantasy.cricket.football.kabaddi', 'in.myteam11.store', 'com.fantasycricketapp.ballebaazi', 'com.games24x7.my11circle.fantasycricket', 'com.gamezy.lite', 'com.ruby.rummy', 'com.howzat.fantasycricketcashgame']
    # flist,clist = driver_2(['com.trade.groww'],id_database)

    app_database = id_database[0:15]
    app_inputs = ['com.trade.groww','com.kakaopay.app','com.thebuilder.fantasypredictor','com.indpay.getcash','com.kaleidofin.kaleidofinCustomer', 'org.altruist.BajajExperia', 'com.gopaysense.android.boost']
    flist,clist = driver_2(app_inputs,app_database)
    
    print(flist,clist)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
